In [11]:
import os

# Setup Paths

In [28]:
WORKSPACE_PATH = r"Tensorflow/workspace"
SCRIPTS_PATH = r"Tensorflow/scripts"
PRE_PROCESSING_SCRIPTS_PATHS = os.path.join(SCRIPTS_PATH, 'preprocessing')
APIMODEL_PATH = r"Tensorflow/models"
CUSTOM_MODEL_NAME = "my_ssd_mobilenet_320"
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
ANNOTATION_PATH = WORKSPACE_PATH + "/training_demo/annotations"
IMAGE_PATH = WORKSPACE_PATH + "/training_demo/images"
MODEL_PATH = WORKSPACE_PATH + "/training_demo/models"
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + "/training_demo/pre-trained-models"
CONFIG_PATH = MODEL_PATH + "/my_ssd_resnet50_v1_fpn/pipeline.config"
CHECKPOINT_PATH = MODEL_PATH + "/my_ssd_mobilenet_320/"
OUTPUT_PATH = os.path.join(MODEL_PATH, CUSTOM_MODEL_NAME, 'export')
TFJS_PATH = os.path.join(MODEL_PATH, CUSTOM_MODEL_NAME, 'tfjsexport')
TFLITE_PATH = os.path.join(MODEL_PATH, CUSTOM_MODEL_NAME, 'tfliteexport')
PROTOC_PATH = os.path.join('Tensorflow', 'protoc')
OUTPUT_PATH = os.path.join(MODEL_PATH, CUSTOM_MODEL_NAME, 'export')
CUSTOM_MODEL_PATH = os.path.join(MODEL_PATH, CUSTOM_MODEL_NAME)
EXPORT_PATH = os.path.join(CUSTOM_MODEL_PATH, 'export')
SAVED_MODEL_PATH = os.path.join(EXPORT_PATH, 'saved_model')
LABELMAP_PATH = os.path.join(ANNOTATION_PATH, "label_map.pbtxt")
PB_FILE_PATH = os.path.join(EXPORT_PATH, 'saved_model', 'saved_model.pb')

# Download dataset from kaggle

In [21]:
FACEMASK_DATASET_URL = 'https://www.kaggle.com/andrewmvd/face-mask-detection/download'
print(f"Download kaggle dataset from this url:{FACEMASK_DATASET_URL}; unzip it and and paste images and annotations directories into {IMAGE_PATH} dir")

Download kaggle dataset from this url:https://www.kaggle.com/andrewmvd/face-mask-detection/download; unzip it and and paste images and annotations directories into Tensorflow/workspace/training_demo/images dir


## Merging annotations and images into faceMaskDataset directory

In [41]:
if os.name =='posix':
    # Creating faceMaskDataset directory inside images directory
    !mkdir {IMAGE_PATH + "/faceMaskDataset"}
    # moving annotations and images into faceMaskDataset directory
    !cp {IMAGE_PATH + "/annotations/."} {IMAGE_PATH + "/faceMaskDataset"} -r
    !cp {IMAGE_PATH + "/images/."} {IMAGE_PATH + "/faceMaskDataset"} -r

## Partioning dataset into train and test directory

In [42]:
!python {PRE_PROCESSING_SCRIPTS_PATHS + "/partition_dataset.py"} -x -i {IMAGE_PATH + "/faceMaskDataset"} -r 0.2
!mv {IMAGE_PATH + "/faceMaskDataset/train"} {IMAGE_PATH}
!mv {IMAGE_PATH + "/faceMaskDataset/test"} {IMAGE_PATH}

Tensorflow/scripts/preprocessing/partition_dataset.py:35: DeprecationWarning: Flags not at the start of the expression '([a-zA-Z0-9\\s_\\\\.\\-\\' (truncated)
  if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(?i)(.jpg|.jpeg|.png)$', f)]


# Create Label Map

In [ ]:
labels = [{'name':'without_mask', 'id':1}, 
          {'name':'with_mask', 'id':2}, 
          {'name':'mask_weared_incorrect', 'id':3}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Create TF records

In [ ]:
!python {SCRIPTS_PATH + "/preprocessing/generate_tfrecord.py"} -x {IMAGE_PATH + "/train"} -l {ANNOTATION_PATH + "/label_map.pbtxt"} -o {ANNOTATION_PATH + "/train.record"}

In [ ]:
!python {SCRIPTS_PATH + "/preprocessing/generate_tfrecord.py"} -x {IMAGE_PATH + "/test"} -l {ANNOTATION_PATH + "/label_map.pbtxt"} -o {ANNOTATION_PATH + "/test.record"}

# Download TF Models Pretrained models from Tensorflow model zoo and install TFOD

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
if not os.path.exists(os.path.join(APIMODEL_PATH, 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
## Install Tensorflow Object Detection

# For Mac and Linux (eg. Ubuntu)
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
# For windows
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

In [10]:
VERIFICATION_SCRIPT = os.path.join(APIMODEL_PATH, 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2021-08-31 15:40:23.022333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /lib:/usr/lib:/usr/local/lib
2021-08-31 15:40:23.022364: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-31 15:40:23.022379: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (akshat-ThinkPad-T14-Gen-1): /proc/driver/nvidia/version does not exist
Running tests under Python 3.8.5: /home/akshat/anaconda3/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-08-31 15:40:23.028391: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable 

I0831 15:40:26.666915 140349715502912 ssd_efficientnet_bifpn_feature_extractor.py:142] EfficientDet EfficientNet backbone version: efficientnet-b0
I0831 15:40:26.667031 140349715502912 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet BiFPN num filters: 64
I0831 15:40:26.667070 140349715502912 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num iterations: 3
I0831 15:40:26.668856 140349715502912 efficientnet_model.py:147] round_filter input=32 output=32
I0831 15:40:26.685400 140349715502912 efficientnet_model.py:147] round_filter input=32 output=32
I0831 15:40:26.685508 140349715502912 efficientnet_model.py:147] round_filter input=16 output=16
I0831 15:40:26.743801 140349715502912 efficientnet_model.py:147] round_filter input=16 output=16
I0831 15:40:26.743917 140349715502912 efficientnet_model.py:147] round_filter input=24 output=24
I0831 15:40:26.920719 140349715502912 efficientnet_model.py:147] round_filter input=24 output=24
I0831 15:40:26.920839 140

I0831 15:40:30.066357 140349715502912 efficientnet_model.py:147] round_filter input=16 output=16
I0831 15:40:30.066518 140349715502912 efficientnet_model.py:147] round_filter input=24 output=24
I0831 15:40:30.328916 140349715502912 efficientnet_model.py:147] round_filter input=24 output=24
I0831 15:40:30.329164 140349715502912 efficientnet_model.py:147] round_filter input=40 output=48
I0831 15:40:30.591856 140349715502912 efficientnet_model.py:147] round_filter input=40 output=48
I0831 15:40:30.592017 140349715502912 efficientnet_model.py:147] round_filter input=80 output=88
I0831 15:40:30.958198 140349715502912 efficientnet_model.py:147] round_filter input=80 output=88
I0831 15:40:30.958417 140349715502912 efficientnet_model.py:147] round_filter input=112 output=120
I0831 15:40:31.311634 140349715502912 efficientnet_model.py:147] round_filter input=112 output=120
I0831 15:40:31.311756 140349715502912 efficientnet_model.py:147] round_filter input=192 output=208
I0831 15:40:31.745949 14

I0831 15:40:36.562308 140349715502912 efficientnet_model.py:147] round_filter input=112 output=160
I0831 15:40:36.562428 140349715502912 efficientnet_model.py:147] round_filter input=192 output=272
I0831 15:40:37.375178 140349715502912 efficientnet_model.py:147] round_filter input=192 output=272
I0831 15:40:37.375439 140349715502912 efficientnet_model.py:147] round_filter input=320 output=448
I0831 15:40:37.698282 140349715502912 efficientnet_model.py:147] round_filter input=1280 output=1792
I0831 15:40:37.769499 140349715502912 efficientnet_model.py:458] Building model efficientnet with params ModelConfig(width_coefficient=1.4, depth_coefficient=1.8, resolution=380, dropout_rate=0.4, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=(1, 1), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=16, output_filters=24, kernel_size=3, num_repeat=2, expand_ratio=6, strides=(2, 2), se_rati

I0831 15:40:45.231473 140349715502912 ssd_efficientnet_bifpn_feature_extractor.py:142] EfficientDet EfficientNet backbone version: efficientnet-b7
I0831 15:40:45.231595 140349715502912 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet BiFPN num filters: 384
I0831 15:40:45.231634 140349715502912 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num iterations: 8
I0831 15:40:45.233090 140349715502912 efficientnet_model.py:147] round_filter input=32 output=64
I0831 15:40:45.244945 140349715502912 efficientnet_model.py:147] round_filter input=32 output=64
I0831 15:40:45.245060 140349715502912 efficientnet_model.py:147] round_filter input=16 output=32
I0831 15:40:45.429523 140349715502912 efficientnet_model.py:147] round_filter input=16 output=32
I0831 15:40:45.429644 140349715502912 efficientnet_model.py:147] round_filter input=24 output=48
I0831 15:40:45.851139 140349715502912 efficientnet_model.py:147] round_filter input=24 output=48
I0831 15:40:45.851266 14

In [ ]:
!pip install tensorflow --upgrade

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {PRETRAINED_MODEL_PATH}
    !cd {PRETRAINED_MODEL_PATH} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    !wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {PRETRAINED_MODEL_PATH} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# Copy model config to training folder

In [3]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [23]:
CONFIG_PATH = f"{MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config"
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [24]:
config

{'model': ssd {
   num_classes: 3
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [13]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [25]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH + '/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [
    ANNOTATION_PATH + '/train.record'
]
pipeline_config.eval_input_reader[
    0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [
    ANNOTATION_PATH + '/test.record'
]

In [26]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:
    f.write(config_text)

# Train the model

In [27]:
TRAINING_SCRIPT = os.path.join(APIMODEL_PATH, 'research', 'object_detection',
                               'model_main_tf2.py')

In [28]:
print(
    """python {} --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000"""
    .format(TRAINING_SCRIPT, MODEL_PATH, CUSTOM_MODEL_NAME, MODEL_PATH,
            CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/training_demo/models/my_ssd_mobilenet_320 --pipeline_config_path=Tensorflow/workspace/training_demo/models/my_ssd_mobilenet_320/pipeline.config --num_train_steps=5000


# Evaluate the model

In [29]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, CHECKPOINT_PATH, CONFIG_PATH, CHECKPOINT_PATH)

In [30]:
command

'python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/training_demo/models/my_ssd_mobilenet_320/ --pipeline_config_path=Tensorflow/workspace/training_demo/models/my_ssd_mobilenet_320/pipeline.config --checkpoint_dir=Tensorflow/workspace/training_demo/models/my_ssd_mobilenet_320/'

In [ ]:
!{command}

2021-08-30 11:58:00.548671: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /lib:/usr/lib:/usr/local/lib
2021-08-30 11:58:00.548702: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-30 11:58:00.548720: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (akshat-ThinkPad-T14-Gen-1): /proc/driver/nvidia/version does not exist
W0830 11:58:00.555963 139814708983616 model_lib_v2.py:1081] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0830 11:58:00.556217 139814708983616 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0830 11:58:00.556316 139814708983616 conf

INFO:tensorflow:Performing evaluation on 86 images.
I0830 11:58:36.992327 139814708983616 coco_evaluation.py:293] Performing evaluation on 86 images.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
I0830 11:58:36.995740 139814708983616 coco_tools.py:116] Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.01s)
I0830 11:58:37.002311 139814708983616 coco_tools.py:138] DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.86s).
Accumulating evaluation results...
DONE (t=0.11s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.417
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.685
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.451
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.339
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100

# Load train model from checkpoint

In [31]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [32]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [33]:
import cv2
import numpy as np

In [34]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

# Real time detection from web cam

In [35]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [36]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

ValueError: in user code:

    <ipython-input-20-d9a72707eda8>:11 detect_fn  *
        image, shapes = detection_model.preprocess(image)
    /home/akshat/anaconda3/lib/python3.8/site-packages/object_detection/meta_architectures/ssd_meta_arch.py:484 preprocess  *
        normalized_inputs, self._image_resizer_fn)
    /home/akshat/anaconda3/lib/python3.8/site-packages/object_detection/utils/shape_utils.py:492 resize_images_and_return_shapes  *
        outputs = static_or_dynamic_map_fn(
    /home/akshat/anaconda3/lib/python3.8/site-packages/object_detection/utils/shape_utils.py:246 static_or_dynamic_map_fn  *
        outputs = [fn(arg) for arg in tf.unstack(elems)]
    /home/akshat/anaconda3/lib/python3.8/site-packages/object_detection/core/preprocessor.py:3327 resize_image  *
        new_image = tf.image.resize_images(
    /home/akshat/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    /home/akshat/anaconda

# Freezing the graph

In [30]:
FREEZE_SCRIPT = os.path.join(APIMODEL_PATH, 'research', 'object_detection', 'exporter_main_v2.py')

In [31]:
command = f"python {FREEZE_SCRIPT} --input_type=image_tensor --pipeline_config_path={CONFIG_PATH} --trained_checkpoint_dir={CHECKPOINT_PATH} --output_directory={OUTPUT_PATH}"

In [32]:
print(command)

python Tensorflow/models/research/object_detection/exporter_main_v2.py --input_type=image_tensor --pipeline_config_path=Tensorflow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir=Tensorflow/workspace/training_demo/models/my_ssd_mobilenet_320/ --output_directory=Tensorflow/workspace/models/my_ssd_mobilenet_320/export


In [33]:
!{command}

2021-08-30 13:07:08.352550: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /lib:/usr/lib:/usr/local/lib
2021-08-30 13:07:08.352580: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-30 13:07:08.352596: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (akshat-ThinkPad-T14-Gen-1): /proc/driver/nvidia/version does not exist
2021-08-30 13:07:08.362216: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
back_prop=False is depr


Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c2fe490> and <keras.layers.core.Lambda object at 0x7f256c29d910>).
W0830 13:07:10.922842 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c2fe490> and <keras.layers.core.Lambda object at 0x7f256c29d910>).

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7f256c2e8730> and <keras.layers.convolutional.Conv2D object at 0x7f256c278190>).
W0830 13:07:10.927159 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable


Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c27e9a0> and <keras.layers.convolutional.Conv2D object at 0x7f256c28e2b0>).
W0830 13:07:10.973399 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c27e9a0> and <keras.layers.convolutional.Conv2D object at 0x7f256c28e2b0>).

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7f256c325c10> and <keras.layers.core.Lambda object at 0x7f256c255a00>).
W0830 13:07:10.977896 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the 


Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7f256c4130a0> and <keras.layers.convolutional.Conv2D object at 0x7f256c569040>).
W0830 13:07:11.025730 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7f256c4130a0> and <keras.layers.convolutional.Conv2D object at 0x7f256c569040>).

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c415760> and <keras.layers.merge.Add object at 0x7f256c424520>).
W0830 13:07:11.030694 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this


Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c298850> and <keras.layers.core.Lambda object at 0x7f256c3ed1f0>).
W0830 13:07:11.076215 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c298850> and <keras.layers.core.Lambda object at 0x7f256c3ed1f0>).

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c2a4a90> and <keras.layers.convolutional.Conv2D object at 0x7f256c2558e0>).
W0830 13:07:11.079364 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Py


Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c387910> and <keras.layers.convolutional.Conv2D object at 0x7f256c25cf40>).
W0830 13:07:11.128112 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c387910> and <keras.layers.convolutional.Conv2D object at 0x7f256c25cf40>).

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7f256c387670> and <object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7f256c59f400>).
W0830 13:07:11.131333 139800139134784 base.py:316] Inconsistent references when loading the checkpoin


Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c561b50> and <object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7f256c5b2220>).
W0830 13:07:11.179067 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c561b50> and <object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7f256c5b2220>).

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7f256c29a0d0> and <keras.layers.core.Lambda object at 0x7f256c25c7c0>).
W0830 13:07:11.182011 139800139134784 base.py:316] Inconsistent references when lo


Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c4c7ca0> and <keras.layers.merge.Add object at 0x7f256c561190>).
W0830 13:07:11.228300 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c4c7ca0> and <keras.layers.merge.Add object at 0x7f256c561190>).

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7f256c287a30> and <keras.layers.core.Lambda object at 0x7f256c29a130>).
W0830 13:07:11.231135 139800139134784 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program 

Traceback (most recent call last):
  File "Tensorflow/models/research/object_detection/exporter_main_v2.py", line 165, in <module>
    app.run(main)
  File "/home/akshat/anaconda3/lib/python3.8/site-packages/absl/app.py", line 303, in run
    _run_main(main, args)
  File "/home/akshat/anaconda3/lib/python3.8/site-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "Tensorflow/models/research/object_detection/exporter_main_v2.py", line 158, in main
    exporter_lib_v2.export_inference_graph(
  File "/home/akshat/anaconda3/lib/python3.8/site-packages/object_detection/exporter_lib_v2.py", line 271, in export_inference_graph
    concrete_function = detection_module.__call__.get_concrete_function()
  File "/home/akshat/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 1233, in get_concrete_function
    concrete = self._get_concrete_function_garbage_collected(*args, **kwargs)
  File "/home/akshat/anaconda3/lib/python3.8/site-packag

# Converting to TFJS

In [34]:
!pip3 install tensorflowjs

     |████████████████████████████████| 64 kB 1.2 MB/s eta 0:00:011
You should consider upgrading via the '/home/akshat/anaconda3/bin/python3 -m pip install --upgrade pip' command.


In [37]:
command = "tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {} {}".format(os.path.join(OUTPUT_PATH, 'saved_model'), TFJS_PATH)

In [38]:
print(command)

tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default Tensorflow/workspace/training_demo/models/my_ssd_mobilenet_320/export/saved_model Tensorflow/workspace/training_demo/models/my_ssd_mobilenet_320/tfjsexport


In [39]:
!{command}

2021-08-30 13:11:28.080397: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /lib:/usr/lib:/usr/local/lib
2021-08-30 13:11:28.080429: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-30 13:11:28.080450: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (akshat-ThinkPad-T14-Gen-1): /proc/driver/nvidia/version does not exist
2021-08-30 13:11:28.080609: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-30 13:11:38.739997: I tensorflow/core/grap

# Converting to TFLite

In [40]:
TFLITE_SCRIPT = os.path.join(APIMODEL_PATH, 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [42]:
command = f"python {TFLITE_SCRIPT} --pipeline_config_path={CONFIG_PATH} --trained_checkpoint_dir={CHECKPOINT_PATH} --output_directory={TFLITE_PATH}"

In [43]:
print(command)

python Tensorflow/models/research/object_detection/export_tflite_graph_tf2.py  --pipeline_config_path=Tensorflow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir=Tensorflow/workspace/training_demo/models/my_ssd_mobilenet_320/ --output_directory=Tensorflow/workspace/training_demo/models/my_ssd_mobilenet_320/tfliteexport


In [45]:
!{command}

2021-08-30 13:20:11.094058: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /lib:/usr/lib:/usr/local/lib
2021-08-30 13:20:11.094095: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-30 13:20:11.094117: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (akshat-ThinkPad-T14-Gen-1): /proc/driver/nvidia/version does not exist
2021-08-30 13:20:11.101287: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.

Two checkpoint references resolved to different o


Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc20209fd0> and <keras.layers.convolutional.Conv2D object at 0x7fdc20223e20>).
W0830 13:20:12.798646 140584839116608 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc20209fd0> and <keras.layers.convolutional.Conv2D object at 0x7fdc20223e20>).

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fdc20172be0> and <keras.layers.core.Lambda object at 0x7fdc20223d30>).
W0830 13:20:12.802804 140584839116608 base.py:316] Inconsistent references when loading the checkpoint into th


Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fdc20065f10> and <keras.layers.convolutional.Conv2D object at 0x7fdc200d9460>).
W0830 13:20:12.850626 140584839116608 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fdc20065f10> and <keras.layers.convolutional.Conv2D object at 0x7fdc200d9460>).

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc201b8bb0> and <keras.layers.core.Lambda object at 0x7fdc201b82b0>).
W0830 13:20:12.855708 140584839116608 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the 


Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fdc201e4700> and <keras.layers.merge.Add object at 0x7fdc20067460>).
W0830 13:20:12.902328 140584839116608 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fdc201e4700> and <keras.layers.merge.Add object at 0x7fdc20067460>).

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc20077130> and <keras.layers.core.Lambda object at 0x7fdc107cd1c0>).
W0830 13:20:12.904745 140584839116608 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object refer


Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc107b0f40> and <keras.layers.core.Lambda object at 0x7fdc107d3eb0>).
W0830 13:20:12.954713 140584839116608 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc107b0f40> and <keras.layers.core.Lambda object at 0x7fdc107d3eb0>).

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fdc200ebd30> and <object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc201d2400>).
W0830 13:20:12.959293 140584839116608 base.py:316] Inconsistent references when loading the


Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc201b4040> and <object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc200b12b0>).
W0830 13:20:13.004987 140584839116608 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc201b4040> and <object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc200b12b0>).

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fdc2007c8b0> and <keras.layers.core.Lambda object at 0x7fdc200de9d0>).
W0830 13:20:13.009382 140584839116608 base.py:316] Inc


Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fdc20144370> and <object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc107d73d0>).
W0830 13:20:13.055815 140584839116608 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fdc20144370> and <object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc107d73d0>).

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc201449d0> and <keras.layers.merge.Add object at 0x7fdc107e2ee0>).
W0830 13:20:13.058983 140584839116608 base.py:316] Inconsistent references when load


Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc201dc910> and <keras.layers.merge.Add object at 0x7fdc200d5220>).
W0830 13:20:13.106074 140584839116608 base.py:316] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fdc201dc910> and <keras.layers.merge.Add object at 0x7fdc200d5220>).

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2D object at 0x7fdc200a7520> and <keras.layers.convolutional.Conv2D object at 0x7fdc20094580>).
W0830 13:20:13.110788 140584839116608 base.py:316] Inconsistent references when loading the checkpoint into this object gra

# Detection from pb file

## Loading model from saved_model.pb file

In [4]:
detection_model = tf.saved_model.load(SAVED_MODEL_PATH)

## Inference function for single model

In [45]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [6]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)
#   print(output_dict)
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.8,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [7]:
def show_inference(model, image_np):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
#   image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)

#   print(category_index)
  # Visualization of the results of a detection.
  final_img =vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks_reframed', None),
          use_normalized_coordinates=True,
          line_thickness=8)
  return(final_img)

## Real time detectin from webcam

In [43]:
import cv2

cap = cv2.VideoCapture(0)
category_index = label_map_util.create_category_index_from_labelmap(LABELMAP_PATH, use_display_name=True)

while 1:
    _,img = cap.read()
    
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    final_img = show_inference(detection_model,img)
    
    final_img = cv2.cvtColor(final_img,cv2.COLOR_RGB2BGR)

    cv2.imshow('Face Mask Detection',final_img)

#     cv2.imshow('img',img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()